In [1]:
from yahoofinancials import YahooFinancials
import yahoo_fin.stock_info as si

In [29]:
'''
- Añadir un método para obtener el ticker de una empresa del SP500, NASDAQ o DOW JONES
'''

'\n- Añadir un método para obtener el ticker de una empresa del SP500, NASDAQ o DOW JONES\n'

In [3]:
# Obtener el ticker de una empresa
ticker= 'MSFT'

In [28]:
stock_obj = YahooFinancials(ticker)

# Obtener el precio de una acción y algunos ratios financieros
price_dict = stock_obj.get_stock_price_data()[ticker]
current = price_dict['regularMarketPreviousClose']
currency = price_dict['currency']
shares_outstandings = stock_obj.get_num_shares_outstanding()
PERatio=stock_obj.get_pe_ratio()
EarningPerShare_dict = stock_obj.get_earnings_per_share()


In [27]:
# Obtener el estimado de crecimiento de los proximos 5 años
Projected_GR = si.get_analysts_info(ticker)['Growth Estimates'][ticker]
print(Projected_GR)

print('-'*50)
df= Projected_GR.to_frame(name='GR')
df['GR']= df['GR'].str.rstrip('%').astype(float)

x=df._get_value(4, 'GR')
print(f'x = {x}')

0    12.80%
1    14.20%
2    11.70%
3    15.10%
4    14.40%
5    21.26%
Name: MSFT, dtype: object
--------------------------------------------------
x = 14.4


In [15]:
# Se pronostica el EPS de los proximos 10 años
earnings_dict = {0:round(stock_obj.get_earnings_per_share(),2)}

for i in range(1,10):

    j = i - 1

    earnings_dict[i] = round(earnings_dict[j]+(earnings_dict[j] * (x/100)),2)

earnings_dict

{0: 9.69,
 1: 11.09,
 2: 12.69,
 3: 14.52,
 4: 16.61,
 5: 19.0,
 6: 21.74,
 7: 24.87,
 8: 28.45,
 9: 32.55}

In [16]:
# Se calcula el precio intrinseco de la acción usando la inducción hacia atras (backward induction) con el ratio P/E
intrinsic_price_dict = {9: earnings_dict[9]*(PERatio)}

for i in range(8,-1,-1):

    j = i + 1

    intrinsic_price_dict[i] = intrinsic_price_dict[j]/(1+0.15)

intrinsic_price_dict

{9: 1084.9329036,
 8: 943.4199161739131,
 7: 820.3651444990548,
 6: 713.3609952165694,
 5: 620.3139088839735,
 4: 539.4033990295421,
 3: 469.0464339387323,
 2: 407.86646429454987,
 1: 354.66649069091295,
 0: 308.40564407905475}

In [21]:
# Se calcula el precio de compra de la acción con un margen de seguridad del 80%
current_intrinsic_price = round(intrinsic_price_dict[0],2)
buyable_price = round((current_intrinsic_price * 0.8), 2)
buyable_price

246.73

In [22]:
# Se compara el precio de compra con el precio actual de la acción
if buyable_price > stock_obj.get_daily_high(): 
    print(f'{ticker}: Cheap\nTarget = <${buyable_price}\nCurrent = ${current}.')

else:
    print(f'{ticker}: Expensive\nTarget = <${buyable_price}\nCurrent = ${current}')

MSFT: Expensive
Target = <$246.73
Current = $319.97
